In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from embeddings import get_embedding_matrix, sentence_to_indexes
from emb_path import glove_6B_300d_path, lexvec_7B_300d_path, glove_6B_50d_path
from sklearn.metrics.pairwise import cosine_similarity
from gem import SentenceEmbedder
from scipy.stats import pearsonr
from utils import read_sts
import os
from tqdm import tqdm_notebook as tqdm
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Prepare datasets

In [2]:
IMDB_PATH = './data/imdb'

IMDB_TRAIN = os.path.join(IMDB_PATH, 'train')
IMDB_TRAIN_POS = os.path.join(IMDB_TRAIN, 'pos')
IMDB_TRAIN_NEG = os.path.join(IMDB_TRAIN, 'neg')

IMDB_TEST = os.path.join(IMDB_PATH, 'test')

In [3]:
train_pos_samples = [os.path.join(IMDB_TRAIN_POS, s) for s in os.listdir(IMDB_TRAIN_POS)]
train_neg_samples = [os.path.join(IMDB_TRAIN_NEG, s) for s in os.listdir(IMDB_TRAIN_NEG)]

test_samples = [os.path.join(IMDB_TEST, s) for s in os.listdir(IMDB_TEST)]

In [4]:
print(len(train_pos_samples), len(train_neg_samples), len(test_samples))

12500 12501 11001


# Process every review

In [5]:
import nltk.data

In [6]:
# nltk.download('punkt')

In [7]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [9]:
pos_reviews = []
neg_reviews = []

try:
    for review in tqdm(train_pos_samples[:]):
        with open(review, 'r', encoding="utf-8") as file:
            data = file.read().replace('\n', '')
            data = data.replace('<br />', ' ')
            data = data.replace('.', '')
    #         data_sentences = tokenizer.tokenize(data)
            pos_reviews += [data]
except UnicodeDecodeError:
    pass
    
try:
    for review in tqdm(train_neg_samples[:]):
        with open(review, 'r', encoding="utf-8") as file:
            data = file.read().replace('\n', '')
            data = data.replace('<br />', ' ')
            data = data.replace('.', '')
    #         data_sentences = tokenizer.tokenize(data)
            neg_reviews += [data]
except UnicodeDecodeError:
    pass

In [10]:
labels = [1]*len(pos_reviews) + [0]*len(neg_reviews)
reviews = pos_reviews + neg_reviews
print(len(reviews))

25000


In [11]:
reviews[0]

'Bromwell High is a cartoon comedy It ran at the same time as some other programs about school life, such as "Teachers" My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers" The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled  at  High A classic line: INSPECTOR: I\'m here to sack one of your teachers STUDENT: Welcome to Bromwell High I expect that many adults of my age think that Bromwell High is far fetched What a pity that it isn\'t!'

# Compute embeddings

In [12]:
embds = {"Glove": get_embedding_matrix(glove_6B_300d_path)}

In [13]:
%%time

embeddings = []

print('Unigrams:\n\n')

for e_name, (e, v) in embds.items():
    model = SentenceEmbedder(reviews, e, v, False)
    result, s = model.gem(sigma_power=3)
    embeddings += [result]

Unigrams:




25000it [22:18, 18.67it/s]
4767it [24:30,  3.24it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

25000it [2:09:01,  3.23it/s]

CPU times: user 13h 48min 37s, sys: 1d 18h 1min 28s, total: 2d 7h 50min 5s
Wall time: 2h 31min 26s


In [16]:
embeddings[0].shape

(25000, 300)

In [17]:
np.save('imdb_embeddings.npy', embeddings[0])

In [18]:
imdb_embeddings = np.load('imdb_embeddings.npy')

In [22]:
train_imdb = np.append(imdb_embeddings[:10000], imdb_embeddings[12500:22500], axis=0)
train_imdb_labels = np.array([1]*10000+[0]*10000)

test_imdb = np.append(imdb_embeddings[10000:12500], imdb_embeddings[22500:], axis=0)
test_imdb_labels = np.array([1]*2500+[0]*2500)

# Fit&Predict

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(train_imdb, train_imdb_labels)